In [1]:
#
#    Notebook de cours MAP412 - Chapitre 8 - M. Massot 2020-2021 - Ecole polytechnique
#    ----------   
#    Limites des méthodes d'ordre 1
#    
#    Auteurs : L. Séries et M. Massot - (C) 2021
#    

# Limites des méthodes d'ordre 1

In [2]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import root
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"

## Explosion thermique

Nous considérons un réacteur chimique homogène décrit par l’évolution d’une température adimensionnée. La variable $\widetilde{\theta}$ vaut zéro au temps initial, elle est monotone croissante et, suite à un emballement thermique, elle va brutalement s’élever et atteindre la température adiabatique de flamme adimensionnée ${\overline T_r}$ en fin de réaction. Sous certaines hypothèses sur la réaction, on arrive à une équation différentielle de la forme suivante :

$$
d_{\tau} \widetilde{\theta} = f(\widetilde{\theta}) = \exp\Bigg(\frac{\widetilde{\theta}}{1 + \widetilde{\theta} \, / \, \beta}\Bigg) \Bigg(1 - \frac{\widetilde{\theta}}{\overline T_r} \Bigg)
$$

où $\beta$ est directement reliée à l’énergie d’activation ($\beta = \frac{E_a}{RT_0}$) où $E_a$ est l’énergie d’activation de la réaction, $R$ la contante des gaz parfaits et $T_0$ la température initiale dimensionnelle.

In [3]:
class explosion_with_consumption_1eq_model:

    def __init__(self, Tr, beta):
        self.Tr = Tr
        self.beta = beta 

    def fcn(self, t, theta):
        beta = self.beta
        Tr = self.Tr
        theta_dot = np.exp(theta/(1+(theta/beta))) * (1-theta/Tr) 
        return theta_dot
    
    def jac(self, t, theta):
        beta = self.beta
        Tr = self.Tr
        exp = np.exp(theta/(1+(theta/beta)))
        return (exp/(1+theta/beta)**2) * (1-theta/Tr) - exp/Tr

## Méthodes d'ordre 1

In [4]:
#####################################################
class ode_result:
    def __init__(self, y, t):
        self.y = y
        self.t = t

#####################################################
def forward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    #y = np.zeros((neq, nt), order='F')
    y = np.zeros((neq, nt))
    y[:,0] = yini

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*np.array(fcn(tn, yn))

    return ode_result(y, t)

#####################################################
def backward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    #y = np.zeros((neq, nt), order='F')
    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini

    def g(uip1, *args):
        uip, tip1 = args
        return uip1 - uip - dt*np.array(fcn(tip1, uip1))

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y0 = yn + dt*np.array(fcn(tn, yn))
        # solve y[:,it+1] - y[:,it] - dt * fcn(tini + (it+1)*dt, y[:,it+1]) = 0
        sol = root(g, y0, (yn, tn+dt))
        y[:,it+1] = sol.x

    return ode_result(y, t)

In [7]:
Tr = 200.0
beta = 10. 

emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
fcn = emwc1eq.fcn
jac = emwc1eq.jac

tini = 0.0
tend = 5.0
nt = np.array([171, 201, 401, 501, 1001, 2001])

yini = (0.,)

fig =  make_subplots(rows=3, cols=1, vertical_spacing=0.1,
                     subplot_titles=("Solution", "Eigen value", "Global error"))

for i, nt_i in enumerate(nt):
    
    dt = (tend-tini) / (nt_i-1)

    emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
    fcn = emwc1eq.fcn
    jac = emwc1eq.jac

    sol_ref = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-9, rtol=1.e-9)
    sol_fe = forward_euler(tini, tend, nt_i, yini, fcn)
    sol_be = backward_euler(tini, tend, nt_i, yini, fcn)
    sol_tmp = solve_ivp(fcn, (tini, tend), yini, method='Radau', t_eval=sol_fe.t, atol=1.e-9, rtol=1.e-9)
    err_fe = np.abs(sol_tmp.y[0] - sol_fe.y[0])
    err_be = np.abs(sol_tmp.y[0] - sol_be.y[0])

    fig.add_trace(go.Scatter(visible=False, x=sol_ref.t, y=sol_ref.y[0],  line_color='rgb(76,114,176)', legendgroup='sol', 
                         legendgrouptitle_text="Solution", name='ref sol'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=sol_fe.y[0], line_color='rgb(221,132,82)', legendgroup='sol', 
                         legendgrouptitle_text="Solution", name='forward euler'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_be.t, y=sol_be.y[0], line_color='rgb(85,168,104)', legendgroup='sol', 
                         legendgrouptitle_text="Solution", name='backward euler'), row=1, col=1)

    eig_vals_ref = np.zeros(sol_ref.t.size)
    for it in range(sol_ref.t.size):
        eig_vals_ref[it] = jac(0, sol_ref.y[:,it])
    eig_vals_fe = np.zeros(sol_fe.t.size)
    for it in range(sol_be.t.size):
        eig_vals_fe[it] = jac(0, sol_fe.y[:,it])
    eig_vals_be = np.zeros(sol_be.t.size)
    for it in range(sol_be.t.size):
        eig_vals_be[it] = jac(0, sol_be.y[:,it])

    fig.add_trace(go.Scatter(visible=False, x=sol_ref.t, y=eig_vals_ref, line_color='rgb(76,114,176)', legendgroup='eig', 
                         legendgrouptitle_text="Eigen value", name='ref sol'), row=2, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=eig_vals_fe, line_color='rgb(221,132,82)', legendgroup='eig', 
                         legendgrouptitle_text="Eigen value", name='forward euler'), row=2, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_be.t, y=eig_vals_be, line_color='rgb(85,168,104)', legendgroup='eig', 
                         legendgrouptitle_text="Eigen value", name='backward_euler'), row=2, col=1)

    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=err_fe, line_color='rgb(221,132,82)', legendgroup='err', 
                         legendgrouptitle_text="Global error", name='forward euler'), row=3, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_be.t, y=err_be, line_color='rgb(85,168,104)', legendgroup='err', 
                         legendgrouptitle_text="Global error", name='backward euler'), row=3, col=1)

for i in range(3*8, 4*8): fig.data[i].visible = True

# create slider
steps = []
for i, nt_i in enumerate(nt):
    step = dict(method="update", label = f"{nt_i}", 
                args=[{"visible": [(el==8*i) or (el==8*i+1) or (el==8*i+2) or (el==8*i+3) or (el==8*i+4) or (el==8*i+5)
                                   or (el==8*i+6) or (el==8*i+7)  for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(active=3, currentvalue={'prefix': 'nt = '}, steps=steps)]

fig.update_layout(sliders=sliders, height=1000, legend_tracegroupgap=210,)
fig.show()

La convergence très lente de l'erreur vers zéro nous convaint qu'il n'est pas raisonnable de poursuivre avec une méthode aussi peu préciser et d'aller voir les méthodes d'ordre plus élevée permettant une convergence plus rapide v